In [1]:
import pyspark
from delta import *
builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
        .config("spark.jars.packages", "")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
        .config("spark.databricks.delta.retentionDurationCheck.enabled","false")
spark = configure_spark_with_delta_pip(builder).getOrCreate()

your 131072x1 screen size is bogus. expect trouble


23/04/21 23:26:34 WARN Utils: Your hostname, DESKTOP-1LBECNS resolves to a loopback address: 127.0.1.1; using 192.168.255.35 instead (on interface eth0)
23/04/21 23:26:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/aleks/git/my-projects/rust-playground/delta-rs/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/aleks/.ivy2/cache
The jars for the packages stored in: /home/aleks/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bd40fdac-efac-440b-abce-2591696138a4;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 194ms :: artifacts dl 8ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0  

23/04/21 23:26:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:

table_path = "./tmp/delta-table"
df = spark.read.format("delta").load(table_path)
df.show()

23/04/21 23:26:55 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+
| id|
+---+
|352|
| 46|
| 84|
|220|
|127|
|191|
| 73|
|150|
|292|
|401|
| 52|
|390|
|253|
|229|
|208|
| 49|
|216|
| 89|
| 71|
| 79|
+---+
only showing top 20 rows



In [31]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forPath(spark, table_path)

# Update every even value by adding 100 to it
deltaTable.update(
  condition = expr("id % 2 == 0"),
  set = { "id": expr("id + 100") })

# Delete every even value
deltaTable.delete(condition = expr("id % 2 == 0"))

# Upsert (merge) new data
newData = spark.range(0, 20)

deltaTable.alias("oldData") \
  .merge(
    newData.alias("newData"),
    "oldData.id = newData.id") \
  .whenMatchedUpdate(set = { "id": col("newData.id") }) \
  .whenNotMatchedInsert(values = { "id": col("newData.id") }) \
  .execute()

deltaTable.toDF().show()

23/04/19 01:33:08 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
23/04/19 01:33:08 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
23/04/19 01:33:08 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+



In [36]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, table_path)

fullHistoryDF = deltaTable.history()    # get the full history of the table
fullHistoryDF.show()
#lastOperationDF = deltaTable.history(1) # get the last operation

+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|   operation| operationParameters| job|notebook|clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|     14|2023-04-19 01:33:...|  null|    null|       MERGE|{predicate -> (ol...|null|    null|     null|         13|     Serializable|        false|{numTargetRowsCop...|        null|Apache-Spark/3.3....|
|     13|2023-04-19 01:33:...|  null|    null|      DELETE|{predicate -> ["(...|null|    null|     null|         12|     Serializable|        false|{numRemovedFiles ...|        null|Ap

In [56]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, table_path)

detailDF = deltaTable.detail()
detailDF.show()

+------+--------------------+----+-----------+--------------------+--------------------+--------------------+----------------+--------+-----------+----------+----------------+----------------+--------------------+
|format|                  id|name|description|            location|           createdAt|        lastModified|partitionColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|       tableFeatures|
+------+--------------------+----+-----------+--------------------+--------------------+--------------------+----------------+--------+-----------+----------+----------------+----------------+--------------------+
| delta|70c5aaf5-74e9-46e...|null|       null|file:/home/aleks/...|2023-04-19 01:45:...|2023-04-19 02:00:...|              []|       1|       4357|        {}|               1|               2|[appendOnly, inva...|
+------+--------------------+----+-----------+--------------------+--------------------+--------------------+----------------+--------+---------

In [53]:
df = spark.read.format("delta").option("versionAsOf", 0).load(table_path)
df.show()

+---+
| id|
+---+
+---+



In [55]:

from delta.tables import *
deltaTable = DeltaTable.forPath(spark, "./tmp/delta-table")
deltaTable.optimize().executeCompaction()

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

In [57]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, table_path)  # path-based tables, or
#deltaTable = DeltaTable.forName(spark, tableName)    # Hive metastore-based tables

deltaTable.vacuum(0)        # vacuum files not required by versions older than the default retention period

#deltaTable.vacuum(100)     # vacuum files not required by versions more than 100 hours old

Deleted 915 files and directories in a total of 1 directories.


DataFrame[]

In [23]:
spark.stop()

### STREAMING

In [44]:
streamingDf = spark.readStream.format("rate").load()

stream = streamingDf.selectExpr("value as id").writeStream.format("delta").option("checkpointLocation", "./tmp/checkpoint").start("./tmp/delta-table")


23/04/19 01:45:06 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [51]:
stream.stop()

23/04/19 02:00:20 ERROR FileFormatWriter: Aborting job d0ba25d7-a813-4631-bbc7-bf941d25ca9a.
java.lang.InterruptedException
	at java.base/java.util.concurrent.locks.AbstractQueuedSynchronizer.acquireSharedInterruptibly(AbstractQueuedSynchronizer.java:1048)
	at scala.concurrent.impl.Promise$DefaultPromise.tryAwait(Promise.scala:242)
	at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:258)
	at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:187)
	at org.apache.spark.util.ThreadUtils$.awaitReady(ThreadUtils.scala:334)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:943)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2238)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:255)
	at org.apache.spark.sql.delta.files.TransactionalWrite.$anonfun$writeFiles$1(TransactionalWrite.scala:398)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala

23/04/19 02:00:20 WARN TaskSetManager: Lost task 0.0 in stage 9061.0 (TID 137668) (192.168.250.148 executor driver): TaskKilled (Stage cancelled)
23/04/19 02:00:20 WARN TaskSetManager: Lost task 4.0 in stage 9061.0 (TID 137672) (192.168.250.148 executor driver): TaskKilled (Stage cancelled)
23/04/19 02:00:20 WARN TaskSetManager: Lost task 3.0 in stage 9061.0 (TID 137671) (192.168.250.148 executor driver): TaskKilled (Stage cancelled)
23/04/19 02:00:20 WARN TaskSetManager: Lost task 6.0 in stage 9061.0 (TID 137674) (192.168.250.148 executor driver): TaskKilled (Stage cancelled)
23/04/19 02:00:20 WARN TaskSetManager: Lost task 5.0 in stage 9061.0 (TID 137673) (192.168.250.148 executor driver): TaskKilled (Stage cancelled)
23/04/19 02:00:20 WARN TaskSetManager: Lost task 1.0 in stage 9061.0 (TID 137669) (192.168.250.148 executor driver): TaskKilled (Stage cancelled)
23/04/19 02:00:20 WARN TaskSetManager: Lost task 2.0 in stage 9061.0 (TID 137670) (192.168.250.148 executor driver): TaskKil

In [45]:
df = spark.read.format("delta").load(table_path)
df.show()

+---+
| id|
+---+
| 46|
| 73|
| 52|
| 49|
| 71|
| 79|
| 12|
| 16|
|  4|
| 17|
| 19|
| 56|
| 27|
| 59|
| 53|
| 77|
| 13|
| 37|
| 39|
| 24|
+---+
only showing top 20 rows



In [67]:
p_path = "./tmp/csv_data/person.csv"
personsDf = spark.read.csv(path=p_path,header=True,)
personsDf.show()
personsDf.printSchema()

+-----+---+-------+
| name|age|city_id|
+-----+---+-------+
|aleks| 10|      1|
|burcu| 20|      2|
| maja| 20|      1|
+-----+---+-------+

root
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- city_id: string (nullable = true)



In [6]:
c_path = "./tmp/csv_data/city.csv"
cytyDf = spark.read.csv(path=c_path,header=True)
cytyDf.show()
cytyDf.printSchema()

+-------+---------+
|city_id|city_name|
+-------+---------+
|      1| Belgrade|
|      2| Istanbul|
+-------+---------+

root
 |-- city_id: string (nullable = true)
 |-- city_name: string (nullable = true)



In [10]:
personsDf.filter(personsDf["age"]> 15).select(personsDf["name"]).show()

+-----+
| name|
+-----+
|burcu|
| maja|
+-----+



In [11]:
personsDf.groupBy(personsDf["city_id"]).count().show()

+-------+-----+
|city_id|count|
+-------+-----+
|      1|    2|
|      2|    1|
+-------+-----+



In [12]:
personsDf.join(cytyDf, personsDf["city_id"] == cytyDf["city_id"]).show()

+-----+---+-------+-------+---------+
| name|age|city_id|city_id|city_name|
+-----+---+-------+-------+---------+
|aleks| 10|      1|      1| Belgrade|
|burcu| 20|      2|      2| Istanbul|
| maja| 20|      1|      1| Belgrade|
+-----+---+-------+-------+---------+



In [13]:
personsDf.write.format("delta").save("./tmp/persons-delta")

In [19]:
p_df = spark.read.format("delta").load("./tmp/persons-delta/")
#p_df.show()
p_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- city_id: string (nullable = true)



In [73]:
p_df2 = spark.read.csv("./tmp/csv_data/person_new.csv",header=True)
p_df2.show()

+------+---+-------+
|  name|age|city_id|
+------+---+-------+
|zeljko| 25|      3|
| burcu| 18|      2|
|  nata| 13|      1|
+------+---+-------+



In [23]:
personsDf.show()


+-----+---+-------+
| name|age|city_id|
+-----+---+-------+
|aleks| 10|      1|
|burcu| 20|      2|
| maja| 20|      1|
+-----+---+-------+



In [25]:
personsDf.write.saveAsTable("persons")

In [31]:
spark.sql('DESCRIBE DETAIL persons').select("location").show(20,False)

+---------------------------------------------------------------------------------+
|location                                                                         |
+---------------------------------------------------------------------------------+
|file:/home/aleks/git/my-projects/rust-playground/delta-rs/spark-warehouse/persons|
+---------------------------------------------------------------------------------+



In [32]:
delta_T = DeltaTable.forPath(spark, "./tmp/persons-delta/") 
delta_T

In [76]:
delta_T.toDF().show()

+------+---+-------+
|  name|age|city_id|
+------+---+-------+
| burcu| 18|      2|
|  maja| 25|      1|
|  nata| 13|      1|
|zeljko| 25|      3|
+------+---+-------+



In [75]:
delta_T.history().show(20,False)

+-------+-----------------------+------+--------+---------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation|operationParameters                                                           

In [77]:
p_df2.show()

+------+---+-------+
|  name|age|city_id|
+------+---+-------+
|zeljko| 25|      3|
| burcu| 18|      2|
|  nata| 13|      1|
+------+---+-------+



In [69]:
p_df.show()

+-----+---+-------+
| name|age|city_id|
+-----+---+-------+
|burcu| 20|      2|
| maja| 25|      1|
+-----+---+-------+



In [74]:
p_dt = DeltaTable.forPath(spark,"./tmp/persons-delta/")
p_dt.alias("old").merge(
    source= p_df2.alias("new"),
    condition= "new.name = old.name"
)\
.whenMatchedUpdateAll()\
.whenNotMatchedInsertAll().execute()

23/04/22 10:06:46 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
23/04/22 10:06:46 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
23/04/22 10:06:46 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.


In [79]:
wiki_stream = spark\
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "https://stream.wikimedia.org/v2/stream/mediawiki.recentchange") \
    .option("subscribe","eqiad.mediawiki.recentchange")\
    .load()

AnalysisException:  Failed to find data source: kafka. Please deploy the application as per the deployment section of "Structured Streaming + Kafka Integration Guide".        